In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import math as mt
import warnings
import stockstats
warnings.filterwarnings('ignore')
import pandas_ta as ta
import plotly.express as px
import plotly.graph_objects as go
import math as mt

In [3]:
COM = yf.download("GLD", "2020-05-21", "2023-01-21")
EQY = yf.download("^GSPC", "2020-05-21", "2023-01-21")
EQY['Date'] = EQY.index
BOND = yf.download("IEF", "2020-05-21", "2023-01-21")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
import pandas as pd

combined_df = pd.DataFrame({
    'Close_EQY': EQY['Close'],
    'Close_BOND': BOND['Close'],
    'Close_COM': COM['Close'],
    'Date': EQY['Date']
})
combined_df.index = combined_df['Date']
combined_df = combined_df.drop(['Date'], axis = 1)

correlation_EQY_BOND = combined_df['Close_EQY'].rolling(window=60).corr(combined_df['Close_BOND'])
correlation_EQY_COM = combined_df['Close_EQY'].rolling(window=60).corr(combined_df['Close_COM'])


In [5]:
correlation_COM = pd.DataFrame(correlation_EQY_COM)
new_names = {0: 'correlation'}
correlation_COM.columns = [new_names.get(col, col) for col in correlation_COM.columns]
correlation_COM = correlation_COM
correlation_COM.reset_index(inplace = False)

correlation_BOND = pd.DataFrame(correlation_EQY_BOND)
new_names = {0: 'correlation'}
correlation_BOND.columns = [new_names.get(col, col) for col in correlation_BOND.columns]
correlation_BOND = correlation_BOND
correlation_BOND.reset_index(inplace = False)

correlation = pd.DataFrame({
    'Close_EQY': EQY['Close'],
    'Close_COM': COM['Close'],
    'Close_BOND': BOND['Close'],
    'Correlation Equity and Bonds': correlation_COM['correlation'],
    'Correlation Equity and COM': correlation_BOND['correlation']
})

In [6]:
correlation['6_months_high_correlation_EQY_and_Bond'] = correlation['Correlation Equity and Bonds'].rolling(window=130).max()
correlation['6_months_low_correlation_EQY_and_Bond'] = correlation['Correlation Equity and Bonds'].rolling(window=130).min()

In [7]:
import plotly.graph_objects as go

y1 = correlation['Correlation Equity and Bonds']
y2 = correlation['Correlation Equity and COM']
x = correlation.index

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=y1,
    mode='lines+markers', 
    name='Correlation Equity and Bonds',
    line=dict(color='blue', width=2), 
    marker=dict(size=5) 
))

fig.add_trace(go.Scatter(
    x=x,
    y=y2,
    mode='lines+markers',
    name='Correlation Equity and COM',
    line=dict(color='orange', width=2),
    marker=dict(size=5)
))

fig.update_layout(
    title='Rolling Correlation Between Equity, Bonds, and COM',
    title_x=0.5, 
    xaxis_title='Date',
    yaxis_title='Correlation Coefficient',
    legend=dict(x=0.1, y=1.1, orientation='h'), 
    hovermode='x', 
    template='plotly_white' 
)

fig.show()



In [31]:
def backtest_with_hedging(data):
    initial_capital = 1000000
    capital = initial_capital
    portfolio_value = []
    investment_eqy = initial_capital*60/100
    investment_bond = initial_capital*40/100
    investment_com = 0
    stocks_eqy = int(investment_eqy/data['Close_EQY'][0])
    stocks_com = int(investment_com/data['Close_COM'][0])
    stocks_bond = int(investment_bond/data['Close_BOND'][0])
    check_from_date = data.index[0]
    last_check_index = 0
    capital_left = initial_capital - data['Close_EQY'][0]*stocks_eqy- data['Close_COM'][0]*stocks_com- data['Close_BOND'][0]*stocks_bond
    
    for i in range(len(data)):
        
        if (data.index[i] >= check_from_date + pd.Timedelta(days=130)):
            
            check_from_date = data.index[i] + pd.Timedelta(days=22)
            
            if (data['Correlation Equity and Bonds'][i] == data['6_months_high_correlation_EQY_and_Bond'][i] and investment_com*100/capital < 20):

                amount_for_turnover = 2*capital/100
                stocks_bond = stocks_bond - int(amount_for_turnover/data['Close_BOND'][i])
                stocks_com += int(((int(amount_for_turnover/data['Close_BOND'][i]))*data['Close_BOND'][i])/data['Close_COM'][i])
                capital_left += (int(amount_for_turnover/data['Close_BOND'][i]))*data['Close_BOND'][i] - (int(((int(amount_for_turnover/data['Close_BOND'][i]))*data['Close_BOND'][i])/data['Close_COM'][i]))*data['Close_COM'][i]
                investment_com = stocks_com*data['Close_COM'][i]
                
            elif (data['Correlation Equity and Bonds'][i] == data['6_months_low_correlation_EQY_and_Bond'][i]):

                amount_for_turnover = 2*capital/100
                stocks_com -= int(amount_for_turnover/data['Close_COM'][i])
                stocks_bond += int(((int(amount_for_turnover/data['Close_COM'][i]))*data['Close_COM'][i])/data['Close_BOND'][i])
                capital_left += (int(amount_for_turnover/data['Close_COM'][i]))*data['Close_COM'][i] - (int(((int(amount_for_turnover/data['Close_COM'][i]))*data['Close_COM'][i])/data['Close_BOND'][i]))*data['Close_BOND'][i]
                investment_com = stocks_com*data['Close_COM'][i]
                
            capital = stocks_com*data['Close_COM'][i] + stocks_eqy*data['Close_EQY'][i] + stocks_bond*data['Close_BOND'][i] + capital_left
            portfolio_value.append(capital)
            
        else:
            
            capital = stocks_com*data['Close_COM'][i] + stocks_eqy*data['Close_EQY'][i] + stocks_bond*data['Close_BOND'][i] + capital_left
            portfolio_value.append(capital)
            
    data['portfolio value with hedging'] = portfolio_value
    print("Final capital using dynamic hedging:", portfolio_value[-1])
    print("Returns (in %) using dynamic hedging:", (portfolio_value[-1] - initial_capital)*100/initial_capital)
    return (portfolio_value[-1] - initial_capital)*100/initial_capital
            
            

In [32]:
def backtest_without_hedging(data):
    
    initial_capital = 1000000
    capital = initial_capital
    portfolio_value = []
    investment_eqy = initial_capital*60/100
    investment_bond = initial_capital*40/100
    investment_com = 0
    stocks_eqy = int(investment_eqy/data['Close_EQY'][0])
    stocks_bond = int(investment_bond/data['Close_BOND'][0])
    capital_left = initial_capital - data['Close_EQY'][0]*stocks_eqy - data['Close_BOND'][0]*stocks_bond
    
    for i in range(len(data)):
        capital = stocks_eqy*data['Close_EQY'][i] + stocks_bond*data['Close_BOND'][i] + capital_left
        portfolio_value.append(capital)
    data['portfolio value without hedging'] = portfolio_value
    print("Final capital without using dynamic hedging:", portfolio_value[-1])
    print("Returns wihout using dynamic hedging (in %):", (portfolio_value[-1] - initial_capital)*100/initial_capital)
    return (portfolio_value[-1] - initial_capital)*100/initial_capital
    

In [33]:
return_with_hedging = backtest_with_hedging(correlation)
return_without_hedging = backtest_without_hedging(correlation)

Final capital using dynamic hedging: 1135596.5875091553
Returns (in %) using dynamic hedging: 13.559658750915528
Final capital without using dynamic hedging: 1133978.068786621
Returns wihout using dynamic hedging (in %): 13.39780687866211


In [34]:
Results = pd.DataFrame({
    '60/40': [return_without_hedging],
    '60/40 with dynamic gsci': [return_with_hedging]
})
Results.index = ['Returns (in %)']

In [35]:
import plotly.graph_objects as go

y1 = correlation['portfolio value without hedging']
y2 = correlation['portfolio value with hedging']
x = correlation.index

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=y1,
    mode='lines+markers',
    name='Portfolio Value Without Hedging',
    line=dict(color='red', width=2),
    marker=dict(size=5)
))

fig.add_trace(go.Scatter(
    x=x,
    y=y2,
    mode='lines+markers',
    name='Portfolio Value With Hedging',
    line=dict(color='blue', width=2),
    marker=dict(size=5)
))

fig.update_layout(
    title='Portfolio Value Comparison',
    title_x=0.5,
    xaxis_title='Date',
    yaxis_title='Portfolio Value',
    legend=dict(x=0.1, y=1.1, orientation='h'),
    hovermode='x',
    template='plotly_white'
)

fig.show()


In [36]:
Results

,60/40,60/40 with dynamic gsci
Returns (in %),13.397807,13.559659
